In [14]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
from pandas.api.types import CategoricalDtype
import plotly.graph_objects as go
import plotly.express as px
import datetime
from plotly.subplots import make_subplots

from sklearn import preprocessing

from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans
import numpy as np

import datetime


In [15]:
# Loading data
file_path = "https://capstone-crime-bucket.s3.amazonaws.com/Mergedcrimedata.csv"
crime_df = pd.read_csv(file_path)

crime_df.head(10)

/Users/rotation/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,CrimeCode,Description,CrimeDate,Year,Latitude,Longitude,Location,City
0,413,ASSAULT - AGGRAVATED - BATTERY,2018-08-29 23:19:00,2018,42.250405,-71.131737,"(42.25040550, -71.13173740)",1
1,413,ASSAULT - AGGRAVATED - BATTERY,2018-08-27 22:56:00,2018,42.321042,-71.072153,"(42.32104169, -71.07215291)",1
2,111,"MURDER, NON-NEGLIGIENT MANSLAUGHTER",2018-08-22 15:00:00,2018,42.315689,-71.089514,"(42.31568920, -71.08951391)",1
3,2662,BALLISTICS EVIDENCE/FOUND,2018-08-22 15:00:00,2018,42.315689,-71.089514,"(42.31568920, -71.08951391)",1
4,413,ASSAULT - AGGRAVATED - BATTERY,2018-08-19 22:46:00,2018,42.378085,-71.057841,"(42.37808452, -71.05784115)",1
5,3114,INVESTIGATE PROPERTY,2018-08-19 22:46:00,2018,42.378085,-71.057841,"(42.37808452, -71.05784115)",1
6,1501,"WEAPON - FIREARM - CARRYING / POSSESSING, ETC",2018-08-15 23:21:00,2018,42.341870,-71.083722,"(42.34186952, -71.08372190)",1
7,111,"MURDER, NON-NEGLIGIENT MANSLAUGHTER",2018-08-14 09:30:00,2018,42.284140,-71.093051,"(42.28414025, -71.09305138)",1
8,413,ASSAULT - AGGRAVATED - BATTERY,2018-08-12 22:52:00,2018,42.357524,-71.130774,"(42.35752408, -71.13077384)",1
9,413,ASSAULT - AGGRAVATED - BATTERY,2018-08-12 21:18:00,2018,42.301550,-71.062351,"(42.30155019, -71.06235124)",1


In [16]:
# Remove the "CrimeCode" column. 
crime_df.drop(columns=["CrimeCode"], inplace=True)
crime_df.head()


,Description,CrimeDate,Year,Latitude,Longitude,Location,City
0,ASSAULT - AGGRAVATED - BATTERY,2018-08-29 23:19:00,2018,42.250405,-71.131737,"(42.25040550, -71.13173740)",1
1,ASSAULT - AGGRAVATED - BATTERY,2018-08-27 22:56:00,2018,42.321042,-71.072153,"(42.32104169, -71.07215291)",1
2,"MURDER, NON-NEGLIGIENT MANSLAUGHTER",2018-08-22 15:00:00,2018,42.315689,-71.089514,"(42.31568920, -71.08951391)",1
3,BALLISTICS EVIDENCE/FOUND,2018-08-22 15:00:00,2018,42.315689,-71.089514,"(42.31568920, -71.08951391)",1
4,ASSAULT - AGGRAVATED - BATTERY,2018-08-19 22:46:00,2018,42.378085,-71.057841,"(42.37808452, -71.05784115)",1


In [17]:
# Remove rows that have at least 1 null value.
# Drop null rows
crime_df = crime_df.dropna(how = 'any', axis=0)
crime_df


,Description,CrimeDate,Year,Latitude,Longitude,Location,City
0,ASSAULT - AGGRAVATED - BATTERY,2018-08-29 23:19:00,2018,42.250405,-71.131737,"(42.25040550, -71.13173740)",1
1,ASSAULT - AGGRAVATED - BATTERY,2018-08-27 22:56:00,2018,42.321042,-71.072153,"(42.32104169, -71.07215291)",1
2,"MURDER, NON-NEGLIGIENT MANSLAUGHTER",2018-08-22 15:00:00,2018,42.315689,-71.089514,"(42.31568920, -71.08951391)",1
3,BALLISTICS EVIDENCE/FOUND,2018-08-22 15:00:00,2018,42.315689,-71.089514,"(42.31568920, -71.08951391)",1
4,ASSAULT - AGGRAVATED - BATTERY,2018-08-19 22:46:00,2018,42.378085,-71.057841,"(42.37808452, -71.05784115)",1
...,...,...,...,...,...,...,...
1222943,AGG. ASSAULT,01/01/2012,2012,39.285930,-76.585720,"(39.2859300000, -76.5857200000)",3
1222944,COMMON ASSAULT,01/01/2012,2012,39.297370,-76.594030,"(39.2973700000, -76.5940300000)",3
1222945,COMMON ASSAULT,01/01/2012,2012,39.317020,-76.575210,"(39.3170200000, -76.5752100000)",3
1222946,COMMON ASSAULT,01/01/2012,2012,39.276780,-76.614400,"(39.2767800000, -76.6144000000)",3


In [18]:
# Create a new DataFrame that holds only the descriptions.
crime_description_df = crime_df.filter(['Description'], axis = 1)
crime_description_df.head(10)


,Description
0,ASSAULT - AGGRAVATED - BATTERY
1,ASSAULT - AGGRAVATED - BATTERY
2,"MURDER, NON-NEGLIGIENT MANSLAUGHTER"
3,BALLISTICS EVIDENCE/FOUND
4,ASSAULT - AGGRAVATED - BATTERY
5,INVESTIGATE PROPERTY
6,"WEAPON - FIREARM - CARRYING / POSSESSING, ETC"
7,"MURDER, NON-NEGLIGIENT MANSLAUGHTER"
8,ASSAULT - AGGRAVATED - BATTERY
9,ASSAULT - AGGRAVATED - BATTERY


In [19]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crime_df.drop(columns=['Latitude', 'Longitude'], inplace=True)
crime_df.head(10)

,Description,CrimeDate,Year,Location,City
0,ASSAULT - AGGRAVATED - BATTERY,2018-08-29 23:19:00,2018,"(42.25040550, -71.13173740)",1
1,ASSAULT - AGGRAVATED - BATTERY,2018-08-27 22:56:00,2018,"(42.32104169, -71.07215291)",1
2,"MURDER, NON-NEGLIGIENT MANSLAUGHTER",2018-08-22 15:00:00,2018,"(42.31568920, -71.08951391)",1
3,BALLISTICS EVIDENCE/FOUND,2018-08-22 15:00:00,2018,"(42.31568920, -71.08951391)",1
4,ASSAULT - AGGRAVATED - BATTERY,2018-08-19 22:46:00,2018,"(42.37808452, -71.05784115)",1
5,INVESTIGATE PROPERTY,2018-08-19 22:46:00,2018,"(42.37808452, -71.05784115)",1
6,"WEAPON - FIREARM - CARRYING / POSSESSING, ETC",2018-08-15 23:21:00,2018,"(42.34186952, -71.08372190)",1
7,"MURDER, NON-NEGLIGIENT MANSLAUGHTER",2018-08-14 09:30:00,2018,"(42.28414025, -71.09305138)",1
8,ASSAULT - AGGRAVATED - BATTERY,2018-08-12 22:52:00,2018,"(42.35752408, -71.13077384)",1
9,ASSAULT - AGGRAVATED - BATTERY,2018-08-12 21:18:00,2018,"(42.30155019, -71.06235124)",1


In [23]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crime_df, columns=["Description", "CrimeDate"])
X


,Year,Location,City,Description_ABDUCTION - INTICING,Description_ABORTION/ILLEGAL,Description_AFFRAY,Description_AGG. ASSAULT,Description_ARSON,Description_ASSAULT - AGGRAVATED,Description_ASSAULT - AGGRAVATED - BATTERY,...,CrimeDate_2018-08-11 23:14:00,CrimeDate_2018-08-12 21:18:00,CrimeDate_2018-08-12 22:52:00,CrimeDate_2018-08-14 09:30:00,CrimeDate_2018-08-15 16:15:00,CrimeDate_2018-08-15 23:21:00,CrimeDate_2018-08-19 22:46:00,CrimeDate_2018-08-22 15:00:00,CrimeDate_2018-08-27 22:56:00,CrimeDate_2018-08-29 23:19:00
0,2018,"(42.25040550, -71.13173740)",1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,2018,"(42.32104169, -71.07215291)",1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,2018,"(42.31568920, -71.08951391)",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2018,"(42.31568920, -71.08951391)",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2018,"(42.37808452, -71.05784115)",1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222943,2012,"(39.2859300000, -76.5857200000)",3,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1222944,2012,"(39.2973700000, -76.5940300000)",3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1222945,2012,"(39.3170200000, -76.5752100000)",3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1222946,2012,"(39.2767800000, -76.6144000000)",3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#Standardize data with StandardScaler
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled)


ValueError: could not convert string to float: '(42.25040550, -71.13173740)'

In [ ]:
# Initialize PCA model
pca = PCA(n_components=3)

# Get two principal components for the iris data.
crime_pca = pca.fit_transform(X_scaled)

crime_pca

In [ ]:
#Transform PCA data to a dataframe
crime_pcs_df = pd.DataFrame(data = crime_pca, columns = ["PC1", "PC2", "PC3"], index = crime_df.index)
crime_pcs_df.head()

In [ ]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
crime_df_elbow = pd.DataFrame(elbow_data)
crime_df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

In [ ]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(crime_pcs_df)

# Predict clusters
predictions = model.predict(crime_pcs_df)
    
predictions

In [ ]:
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
crime_clustered_df = pd.concat([crime_df, crime_pcs_df], axis = 1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
crime_clustered_df['CoinName'] = crime_name_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
crime_clustered_df['Class'] = predictions

# Print the shape of the clustered_df
print(crime_clustered_df.shape)
crime_clustered_df.head(10)

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    crime_clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name ="Description",
    hover_data=['CrimeDate'],
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [ ]:
# Create a table with tradable cryptocurrencies.
columns = ["Description", "CrimeDate", "Year", "Location", "City", "Class"]
crime_clustered_df.hvplot.table(columns, sortable =True, selectable=True)


In [ ]:
# Print the total number of.
print(f"There are {crime_clustered_df.Description.size} crimes by ___.")



In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
#scaler =MinMaxScaler(feature_range=(0,1))
#cluster_scaled = scaler.fit_transform(clustered_df[["", ""]])
#print(cluster_scaled)


In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
#plot_df = pd.DataFrame(data=clustered_df, columns=['TotalCoinSupply', 'TotalCoinsMined'], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
#plot_df['CoinName'] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
#plot_df['Class'] = clustered_df['Class']

#plot_df.head(10)


In [ ]:

# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="".
#plot_df.hvplot.scatter(x="", y="", by="Class", hover_cols=[""])
